In [1]:
import os
import json
import pandas as pd
from fractions import Fraction
import inflect

import re

In [6]:
DOWNLOAD_DATA_FILEPATH = os.getcwd() + '/data/'
delimiter = '||'

In [7]:
def read_data_from_file(filepath):
    
    with open(filepath, 'r') as f:
        data_dict = json.load(f)
    
    return data_dict

In [8]:
data_list = []

for filename in os.listdir(DOWNLOAD_DATA_FILEPATH):
    if '.json' in filename:
        data = read_data_from_file(DOWNLOAD_DATA_FILEPATH+filename)
        drinks_list = data['drinks']
        if drinks_list is not None:
            for drink in drinks_list:
                data_list.append(drink)

In [9]:
drinks_df = pd.DataFrame(data_list)
drinks_df.sample(3)

,idDrink,strDrink,strDrinkAlternate,strTags,strVideo,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,...,strMeasure10,strMeasure11,strMeasure12,strMeasure13,strMeasure14,strMeasure15,strImageSource,strImageAttribution,strCreativeCommonsConfirmed,dateModified
17,178307,Tequila Slammer,None,Drunk,None,Shot,None,Alcoholic,Hurricane glass,Mix carefully to avoid releasing the dissolved...,...,None,None,None,None,None,None,None,None,No,None
387,13204,New York Lemonade,None,None,None,Cocktail,None,Alcoholic,Cocktail glass,Serve in a chilled cocktail glass. Lemon and s...,...,None,None,None,None,None,None,None,None,No,2017-08-24 10:04:33
371,14888,Zinger,None,None,None,Soft Drink,None,Alcoholic,Highball glass,Get a shot glass and pour in three shots of th...,...,None,None,None,None,None,None,None,None,No,2017-01-28 16:13:34


In [10]:
ingredient_cols = [col for col in drinks_df.columns.values if 'strIngredient' in col]
measure_cols = [col for col in drinks_df.columns.values if 'strMeasure' in col]

In [14]:
remaining_cols = []
for val in drinks_df.columns.values:
    
    if val not in ingredient_cols+measure_cols:
        remaining_cols.append(val)
        
remaining_cols

['idDrink',
 'strDrink',
 'strDrinkAlternate',
 'strTags',
 'strVideo',
 'strCategory',
 'strIBA',
 'strAlcoholic',
 'strGlass',
 'strInstructions',
 'strInstructionsES',
 'strInstructionsDE',
 'strInstructionsFR',
 'strInstructionsIT',
 'strInstructionsZH-HANS',
 'strInstructionsZH-HANT',
 'strDrinkThumb',
 'strImageSource',
 'strImageAttribution',
 'strCreativeCommonsConfirmed',
 'dateModified']

In [291]:
target_cols = []

for ingredient_col, measure_col in zip(ingredient_cols, measure_cols):
    
    col_name = f'{ingredient_col}{delimiter}{measure_col}'
    print(col_name)
    drinks_df[col_name] = drinks_df.apply(lambda x:f'{x[ingredient_col]}{delimiter}{str(x[measure_col])}', axis=1)
    
    target_cols.append(col_name)

strIngredient1||strMeasure1
strIngredient2||strMeasure2
strIngredient3||strMeasure3
strIngredient4||strMeasure4
strIngredient5||strMeasure5
strIngredient6||strMeasure6
strIngredient7||strMeasure7
strIngredient8||strMeasure8
strIngredient9||strMeasure9
strIngredient10||strMeasure10
strIngredient11||strMeasure11
strIngredient12||strMeasure12
strIngredient13||strMeasure13
strIngredient14||strMeasure14
strIngredient15||strMeasure15


In [292]:
ingredients_meta_data = []

for idx, row in drinks_df[target_cols].iterrows():
    
    for col in target_cols:
        
        if 'None' not in row[col].split(delimiter):
            ingredients_meta_data.append({'ingredient': row[col].split(delimiter)[0].strip(),
                                         'measure': row[col].split(delimiter)[1].strip()})

In [293]:
ingredients_df = pd.DataFrame(ingredients_meta_data)

In [294]:
ingredients_df

,ingredient,measure
0,Scotch,1 1/2 oz
1,Wine,1 oz Green Ginger
2,Orange juice,1 oz
3,Dark Rum,1 shot
4,Campari,1 shot
...,...,...
1553,Banana liqueur,1 part
1554,Grenadine,1 part
1555,Pineapple Juice,2 parts
1556,Orange Juice,2 parts


In [295]:
all_ingredients = list(set(list(ingredients_df.ingredient.values)))
len(all_ingredients)

321

In [369]:
sample_filename = os.getcwd() + '/data/sample.txt'

In [370]:
def write_to_file(data, filename=sample_filename):
    
    with open(filename, 'w') as f:
        f.writelines('\n'.join(data))

In [320]:
word_to_float_dict = {' third': '/3',
                     ' fourth': '/4',
                      ' fifth': '/5',
                     ' sixth': '/6',
                      ' seventh': '/7',
                     ' eigth': '/8',
                      ' nineth': '/9',
                     ' tenth': '/10',
                      '½': '1/2'
                     }

target_units = ['or', 'smirnoff']

target_meta_data_flip = ['pinch', 'bottle', 'cube', 'dash', 'splash', 'sprig', 'wedge', 'strip', 'measure', 'leaf']
target_unit_flip = ['fresh']

unit_mapping_dict = {
    'bottles': 'bottle',
'cans': 'can',
'chunks': 'chunk',
'cubes': 'cube',
'cups': 'cup',
'dashes': 'dash',
'drops': 'drop',
'glasses': 'glass',
'inches': 'inch',
'jiggers': 'jigger',
'measures': 'measure',
'packages': 'package',
'parts': 'part',
'pieces': 'piece',
'pinches': 'pinch',
'pints': 'pint',
'shots': 'shot',
'slices': 'slice',
'splashes': 'splash',
'sprigs': 'sprig',
'sticks': 'stick',
'wedges': 'wedge',
'tblsp': 'tbsp',
'scoops': 'scoop',
    'strips': 'strip',
    'leaves': 'leaf'
}

In [350]:
def replace_word_to_float(measurement):
    
    for key, val in word_to_float_dict.items():
        if key in measurement:
            measurement = measurement.replace(key, val)
            break
    return measurement

def transform_fraction(measurement):
    
    fraction_regex = """\d \d\/\d|\d\/\d"""
    measurement = replace_word_to_float(measurement)
    
    fraction = re.findall(fraction_regex, measurement)
    if len(fraction) > 0:
        fraction = fraction[0]
        
        if len(fraction.split()) == 1:
            numeric_val = float(Fraction(fraction))
        elif len(fraction.split()) == 2:
            numeric_val = float(fraction.split()[0]) + float(Fraction(fraction.split()[1]))

        replace_str = str(round(numeric_val, 2))

        return re.sub(fraction_regex, replace_str, measurement)
    else:
        return measurement
    
def check_digit(text):
    
    return text.replace('.','',1).isdigit() or text.replace('-','',1).isdigit()
    
def transform_measurement_tokens(measurement):
    
    measurement = measurement.lower()
    measurement_tokens = measurement.split()
    
    quantity = None
    unit = None
    meta_data = None
    
    if len(measurement_tokens) == 2:
        if check_digit(measurement_tokens[0]) and check_digit(measurement_tokens[1]):
            quantity = measurement_tokens[0]+measurement_tokens[1]
            unit = 'NA'
            meta_data = 'NA'
        elif check_digit(measurement_tokens[0]) and not check_digit(measurement_tokens[1]):
            quantity = measurement_tokens[0]
            unit = measurement_tokens[1]
            meta_data = 'NA'
    elif len(measurement_tokens) == 1:
        if check_digit(measurement_tokens[0]):
            quantity = measurement_tokens[0]
            unit = 'NA'
            meta_data = 'NA'
        elif check_digit(measurement_tokens[0][0]):
            for idx in range(len(measurement_tokens[0])-1, -1, -1):
                if check_digit(measurement_tokens[0][idx]):
                     break
            
            quantity = measurement_tokens[0][:idx+1]
            unit = measurement_tokens[0][idx+1:]
            meta_data = 'NA'
        else:
            quantity = 'NA'
            unit = 'NA'
            meta_data = measurement_tokens[0]
    elif len(measurement_tokens) > 2:
        
        if check_digit(measurement_tokens[0]):
            quantity = measurement_tokens[0]
            unit = measurement_tokens[1]
            meta_data = ' '.join(measurement_tokens[2:])
            if unit in target_units:
                unit = 'NA'
                meta_data = ' '.join(measurement_tokens[1:])
        elif check_digit(measurement_tokens[-1]):
            quantity = measurement_tokens[-1]
            unit = 'NA'
            meta_data = ' '.join(measurement_tokens[:-1])
        else:
            
            for idx, token in enumerate(measurement_tokens):
                
                if check_digit(token):
                    quantity = token
                    unit = measurement_tokens[idx+1]
                    meta_data = ' '.join(measurement_tokens[:-idx-1])
                    break
            
        if quantity is None and unit is None and meta_data is None:
            quantity = 'NA'
            unit = 'NA'
            meta_data = measurement
            
    if quantity is None and unit is None and meta_data is None:
        quantity = 'NA'
        unit = 'NA'
        meta_data = 'NA'
        
    if unit in unit_mapping_dict.keys():
        unit = unit_mapping_dict[unit]
        
    if meta_data in unit_mapping_dict.keys():
        meta_data = unit_mapping_dict[meta_data]
        
    if meta_data in target_meta_data_flip:
        unit, meta_data = meta_data, unit
        
        if quantity is 'NA':
            quantity = str(1)
    elif unit in target_unit_flip:
        unit, meta_data = meta_data, unit
        
    if '-' in quantity:
        min_quantity = float(quantity.split('-')[0])
        max_quantity = float(quantity.split('-')[1])
    elif quantity == 'NA':
        min_quantity = 0.0
        max_quantity = 0.0      
    else:
        min_quantity = 0.0
        max_quantity = float(quantity)

    return (min_quantity, max_quantity, unit, meta_data)

In [359]:
ingredients_df['measurement_fraction_transformed'] = ingredients_df.apply(lambda x:transform_fraction(x['measure']), axis=1)
ingredients_df['measurement_length'] = ingredients_df.apply(lambda x:len(x['measurement_fraction_transformed'].split()), axis=1)

ingredients_df['ingredient_name'] = ingredients_df.apply(lambda x:x['ingredient'].lower(), axis=1)

ingredients_df['measurement_breakdown'] = ingredients_df.apply(lambda x:transform_measurement_tokens(x['measurement_fraction_transformed']), axis=1)

ingredients_df['min_quantity'] = ingredients_df.apply(lambda x:x['measurement_breakdown'][0], axis=1)
ingredients_df['max_quantity'] = ingredients_df.apply(lambda x:x['measurement_breakdown'][1], axis=1)

ingredients_df['unit'] = ingredients_df.apply(lambda x:x['measurement_breakdown'][2], axis=1)
ingredients_df['meta_data'] = ingredients_df.apply(lambda x:x['measurement_breakdown'][3], axis=1)

In [362]:
ingredients_df[ingredients_df['ingredient_name'] == 'Sambuca'.lower()]

,ingredient,measure,measurement_fraction_transformed,measurement_length,measurement_breakdown,quantity,unit,meta_data,min_quantity,max_quantity,ingredient_name
645,Sambuca,1/2 oz,0.5 oz,2,"(0.0, 0.5, oz, NA)",0.5,oz,NA,0.0,0.50,sambuca
686,Sambuca,1 oz,1 oz,2,"(0.0, 1.0, oz, NA)",1,oz,NA,0.0,1.00,sambuca
689,Sambuca,1/4 glass,0.25 glass,2,"(0.0, 0.25, glass, NA)",0.25,glass,NA,0.0,0.25,sambuca
1352,Sambuca,2 cl,2 cl,2,"(0.0, 2.0, cl, NA)",2,cl,NA,0.0,2.00,sambuca
1365,Sambuca,1 1/2 oz,1.5 oz,2,"(0.0, 1.5, oz, NA)",1.5,oz,NA,0.0,1.50,sambuca
1461,Sambuca,1/3 shot,0.33 shot,2,"(0.0, 0.33, shot, NA)",0.33,shot,NA,0.0,0.33,sambuca
1541,Sambuca,1/3 oz,0.33 oz,2,"(0.0, 0.33, oz, NA)",0.33,oz,NA,0.0,0.33,sambuca


In [375]:
def get_insert_names(data):
    
    data = list(set(data))
    names_list = []
    
    for idx, val in enumerate(data):
        names_list.append(str(idx+1) + delimiter + val)
        
    return names_list

In [376]:
ingredients_filename = os.getcwd() + '/sql/ingredients.txt'

ingredient_names_list = get_insert_names(ingredients_df.ingredient_name.values)

write_to_file(ingredient_names_list, ingredients_filename)

In [377]:
measurements_filename = os.getcwd() + '/sql/measurements.txt'

measurements_names_list = get_insert_names(ingredients_df.unit.values)

write_to_file(measurements_names_list, measurements_filename)

In [302]:
ingredients_df.measurement_length.value_counts()

2    1211
1     200
3     133
4       8
5       4
0       1
6       1
Name: measurement_length, dtype: int64

In [378]:
#ingredients_df[ingredients_df['unit'] == 'small']
ingredients_df[ingredients_df['unit'] == 'gr']

,ingredient,measure,measurement_fraction_transformed,measurement_length,measurement_breakdown,quantity,unit,meta_data,min_quantity,max_quantity,ingredient_name
536,Sugar,60 gr,60 gr,2,"(0.0, 60.0, gr, NA)",60,gr,NA,0.0,60.0,sugar
1239,Chocolate,125 gr,125 gr,2,"(0.0, 125.0, gr, NA)",125,gr,NA,0.0,125.0,chocolate


In [337]:
check_digit('8w10')

False

In [353]:
ingredients_df[ingredients_df['min_quantity'] == '8-10']

,ingredient,measure,measurement_fraction_transformed,measurement_length,measurement_breakdown,quantity,unit,meta_data,min_quantity,max_quantity


In [360]:
data = list(set([str(val) for val in ingredients_df.ingredient_name.values]))
#data.sort()
write_to_file(data)

In [163]:
units = ['Dashes', 'Shots', 'Slice', 'Sprig', 'Wedges', 'bottles', 'cans', 'chunks', 'cubes', 'cups', 'dashes', 'drops', 'glasses', 'inches', 'jiggers', 'measures', 'packages', 'parts', 'pieces', 'pinches', 'pints', 'shots', 'slices', 'splashes', 'sprigs', 'sticks', 'wedges']
units = [val.lower() for val in units]
p = inflect.engine()

for unit in units:
    print(p.singular_noun(unit))

In [35]:
import sqlite3

DB_PATH = os.getcwd() + '/sqlite_db/sqlite_cocktail_db.db'

In [36]:
db_conn = sqlite3.connect(DB_PATH)

In [37]:
table_name = 'ingredients'
col_name = 'ingredients_name'
data = 'wine'

In [38]:
#select_query = f'SELECT id FROM {table_name} WHERE {col_name} = ?'
select_query = '''SELECT * FROM cocktail_drinks'''

cursor = db_conn.cursor()
cursor.execute(select_query)

row = cursor.fetchall()
row

[(12388,
  'Thriller',
  None,
  None,
  None,
  'Ordinary Drink',
  None,
  'Alcoholic',
  'Cocktail glass',
  'In a shaker half-filled with ice cubes, combine all of the ingredients. Shake well. Strain into a cocktail glass.',
  'In einem Shaker, der halb mit Eiswürfeln gefüllt ist, alle Zutaten vermengen. Gut schütteln. In ein Cocktailglas abseihen.',
  'https://www.thecocktaildb.com/images/media/drink/rvuswq1461867714.jpg',
  None,
  None,
  'No',
  '2016-04-28 19:21:54')]